# ReadMe de Pagamentos
## Esse script faz:
1. Extração dos .zip dentro da pasta que colocou os arquivos (deve-se colocar dentro da pasta dados_pagamento, coloque no formato .zip)
* Pré-processamento para remover erros e tipos de atributos diferentes
* Insere no banco em uma tabela com tudo (pagamentos_all_bruto)
* Remove arquivos que já não precisa mais
* Preenche as tabelas menores (ação, programa, elemento_despesa...)
* Preenche a tabela oficial de pagamento com as Foreign Key das tabelas menores (Reduz bastante espaço!)
* Remove as tuplas da tabela com tudo, pois não é mais necessária

In [1]:
import pandas as pd
import numpy as np
import zipfile
import os
import psycopg2
import subprocess
import csv

# Comandos para instalar o psycopg2
* Comando para instalar o psycopg2

conda install -c anaconda psycopg2

## Conexão com o Banco de Dados (Só precisa executar apenas uma vez)

In [2]:
try:
    conn = psycopg2.connect(dbname='pagamento_hibrido_db', user='dmining', host='texas.intranet', password='#@dmining#@')
except:
    print("I am unable to connect to the database")

## 1. Extração dos .zip

In [3]:
for file in os.listdir(path='dados_pagamento/'):
    zip_ref = zipfile.ZipFile('dados_pagamento/{}'.format(file), 'r')
    zip_ref.extractall('dados_pagamento/exports')
    zip_ref.close()
os.mkdir('dados_pagamento/processados')

FileNotFoundError: [Errno 2] No such file or directory: 'dados_pagamento/'

## 2. Pré-Processamento dos CSVs

* Foi preenchido com -100 os valores NaN, pois -1 já existia, ou seja, -1 representa os valores que já estavam no csv, enquanto -100 são os valores NaN, existem outros valores negativos, mas não sabe-se o que eles representam.
* Em programa, todos os valores são int, com exceção de um que é 0P que representa 'Sem Informação', esse valor foi convertido para -99 para representar o 0P
* É necessário remover os acentos, pois isso dá problema no primary key com o mesmo código, mas nomes diferentes
    - Atualmente é apenas feito para elemento_despesa, modalidade_despesa, programa.

In [4]:
for file in os.listdir(path='dados_pagamento/exports/'):
    p = pd.read_csv('dados_pagamento/exports/{}'.format(file), delimiter=';',encoding='latin1')
    p['ANO'], p['MES'] = p['Ano e mês do lançamento'].astype(str).str.partition('/')[0], p['Ano e mês do lançamento'].astype(str).str.partition('/')[2]
    p['DATA'] = p['ANO'] + p['MES']
    p = p.drop(['Ano e mês do lançamento'], axis=1)
    p = p.fillna(-100)
    p["Código Grupo de Despesa"] = p["Código Grupo de Despesa"].astype(str).str.replace('I', '-1')
    p = p.astype({"Código Gestão": np.int64, "Código Grupo de Despesa": np.int64})
    p["Valor Empenhado (R$)"] = p["Valor Empenhado (R$)"].astype(str).str.replace(',', '.')
    p["Valor Liquidado (R$)"] = p["Valor Liquidado (R$)"].astype(str).str.replace(',', '.')
    p["Valor Pago (R$)"] = p["Valor Pago (R$)"].astype(str).str.replace(',', '.')
    p["Valor Restos a Pagar Inscritos (R$)"] = p["Valor Restos a Pagar Inscritos (R$)"].astype(str).str.replace(',', '.')
    p["Valor Restos a Pagar Cancelado (R$)"] = p["Valor Restos a Pagar Cancelado (R$)"].astype(str).str.replace(',', '.')
    p["Valor Restos a Pagar Pagos (R$)"] = p["Valor Restos a Pagar Pagos (R$)"].astype(str).str.replace(',', '.')
    p.to_csv('dados_pagamento/processados/{}'.format(file), sep='\t', index=False)
    print('Arquivo Finalizado: {}'.format(file));

FileNotFoundError: [Errno 2] No such file or directory: 'dados_pagamento/exports/'

## 3. Inserção no banco

In [16]:
for file in os.listdir(path='dados_pagamento/processados/'):
    cur = conn.cursor()
    with open('dados_pagamento/processados/' + file, 'r', encoding="utf8") as f:
        next(f)  # Skip the header row.
        cur.copy_from(f, 'pagamentos.pagamento_all_bruto', sep='\t')
    conn.commit()
    print("Importado arquivo de cadastro {} ".format(file))

Importado arquivo de cadastro 201811_Despesas.csv 
Importado arquivo de cadastro 201808_Despesas.csv 
Importado arquivo de cadastro 201801_Despesas.csv 
Importado arquivo de cadastro 201704_Despesas.csv 
Importado arquivo de cadastro 201705_Despesas.csv 
Importado arquivo de cadastro 201809_Despesas.csv 
Importado arquivo de cadastro 201710_Despesas.csv 
Importado arquivo de cadastro 201807_Despesas.csv 
Importado arquivo de cadastro 201706_Despesas.csv 
Importado arquivo de cadastro 201701_Despesas.csv 
Importado arquivo de cadastro 201804_Despesas.csv 
Importado arquivo de cadastro 201703_Despesas.csv 
Importado arquivo de cadastro 201708_Despesas.csv 
Importado arquivo de cadastro 201803_Despesas.csv 
Importado arquivo de cadastro 201707_Despesas.csv 
Importado arquivo de cadastro 201802_Despesas.csv 
Importado arquivo de cadastro 201709_Despesas.csv 
Importado arquivo de cadastro 201805_Despesas.csv 
Importado arquivo de cadastro 201711_Despesas.csv 
Importado arquivo de cadastro 2

## 4. Remoção dos arquivos processados e exportados

In [17]:
c = 0
for file in os.listdir(path='dados_pagamento/exports/'):
        os.remove("dados_pagamento/exports/{}".format(file))
for file in os.listdir(path='dados_pagamento/processados/'):
        os.remove("dados_pagamento/processados/{}".format(file))
os.rmdir("dados_pagamento/exports")
os.rmdir("dados_pagamento/processados")

## 5. Preenchendo as outras tabelas

In [18]:
cur = conn.cursor()
cur.execute("""
INSERT INTO pagamentos.acao
SELECT cod_acao, nome_acao
FROM pagamentos.pagamento_all_bruto;
""")
conn.commit()
print("Ações Finalizado")

cur = conn.cursor()
cur.execute("""
INSERT INTO pagamentos.elemento_despesa
SELECT cod_elemento_despesa, nome_elemento_despesa
FROM pagamentos.pagamento_all_bruto;
""")
conn.commit()
print("Elemento Despesa Finalizado")

cur = conn.cursor()
cur.execute("""
INSERT INTO pagamentos.funcao
SELECT cod_funcao, nome_funcao
FROM pagamentos.pagamento_all_bruto;
""")
conn.commit()
print("Função Finalizado")

cur = conn.cursor()
cur.execute("""
INSERT INTO pagamentos.gestao
SELECT cod_gestao, nome_gestao
FROM pagamentos.pagamento_all_bruto;
""")
conn.commit()
print("Gestão Finalizado")

cur = conn.cursor()
cur.execute("""
INSERT INTO pagamentos.grupo_despesa
SELECT cod_grupo_despesa, nome_grupo_despesa
FROM pagamentos.pagamento_all_bruto;
""")
conn.commit()
print("Grupo Despesa Finalizado")

cur = conn.cursor()
cur.execute("""
INSERT INTO pagamentos.modalidade_despesa
SELECT cod_modalidade_despesa, nome_modalidade_despesa
FROM pagamentos.pagamento_all_bruto;
""")
conn.commit()
print("Modalidade Despesa Finalizado")

cur = conn.cursor()
cur.execute("""
INSERT INTO pagamentos.orgao_superior
SELECT cod_orgao_superior, nome_orgao_superior
FROM pagamentos.pagamento_all_bruto;
""")
conn.commit()
print("Órgão Superior Finalizado")

cur = conn.cursor()
cur.execute("""
INSERT INTO pagamentos.programa
SELECT cod_programa, nome_programa
FROM pagamentos.pagamento_all_bruto;
""")
conn.commit()
print("Programa Finalizado")

cur = conn.cursor()
cur.execute("""
INSERT INTO pagamentos.programa_governo
SELECT cod_programa_governo, nome_programa_governo
FROM pagamentos.pagamento_all_bruto;
""")
conn.commit()
print("Programa_Governo Finalizado")

cur = conn.cursor()
cur.execute("""
INSERT INTO pagamentos.subfuncao
SELECT cod_subfuncao, nome_subfuncao
FROM pagamentos.pagamento_all_bruto;
""")
conn.commit()
print("Subfunção Finalizado")

cur = conn.cursor()
cur.execute("""
INSERT INTO pagamentos.unidade_gestora
SELECT cod_unidade_gestora, nome_unidade_gestora
FROM pagamentos.pagamento_all_bruto;
""")
conn.commit()
print("Unidade Gestora Finalizado")

cur = conn.cursor()
cur.execute("""
INSERT INTO pagamentos.unidade_orcamentaria
SELECT cod_unidade_orcamentaria, nome_unidade_orcamentaria
FROM pagamentos.pagamento_all_bruto;
""")
conn.commit()
print("Unidade Orçamentária Finalizado")

cur = conn.cursor()
cur.execute("""
INSERT INTO pagamentos.orgao_subordinado
SELECT cod_orgao_subordinado, nome_orgao_subordinado, cod_orgao_superior
FROM pagamentos.pagamento_all_bruto;
""")
conn.commit()
print("Órgão Subordinado Finalizado")

Ações Finalizado
Elemento Despesa Finalizado
Função Finalizado
Gestão Finalizado
Grupo Despesa Finalizado
Modalidade Despesa Finalizado
Órgão Superior Finalizado
Programa Finalizado
Programa_Governo Finalizado
Subfunção Finalizado
Unidade Gestora Finalizado
Unidade Orçamentária Finalizado
Órgão Subordinado Finalizado


## 6. Preenchendo a tabela Pagamento com as Foreign Keys

In [19]:
cur = conn.cursor()
cur.execute("""
INSERT INTO pagamentos.pagamento(
            cod_orgao_superior, cod_orgao_subordinado, cod_unidade_gestora, 
            cod_gestao, cod_unidade_orcamentaria, cod_funcao, cod_subfuncao, 
            cod_programa, cod_acao, cod_programa_governo, cod_grupo_despesa, 
            cod_elemento_despesa, cod_modalidade_despesa, valor_empenhado, 
            valor_liquidado, valor_pago, valor_restos_a_pagar_inscritos, 
            valor_restos_a_pagar_cancelado, valor_restos_a_pagar_pagos, mes, 
            ano, data)
SELECT cod_orgao_superior, cod_orgao_subordinado, cod_unidade_gestora, 
            cod_gestao, cod_unidade_orcamentaria, cod_funcao, cod_subfuncao, 
            cod_programa, cod_acao, cod_programa_governo, cod_grupo_despesa, 
            cod_elemento_despesa, cod_modalidade_despesa, valor_empenhado, 
            valor_liquidado, valor_pago, valor_restos_a_pagar_inscritos, 
            valor_restos_a_pagar_cancelado, valor_restos_a_pagar_pagos, mes, 
            ano, data
FROM pagamentos.pagamento_all_bruto
""")

In [20]:
conn.commit()

## 7. Revovendo a tabela com os dados brutos

In [21]:
cur = conn.cursor()
cur.execute("""
TRUNCATE TABLE pagamentos.pagamento_all_bruto
""")

In [22]:
conn.commit()